In [10]:
import pandas as pd
import os
from collections import defaultdict

In [11]:
# load all .csv files from the folder 'data'
root_folder = 'D:/'
files = [f for f in os.listdir(root_folder) if f.endswith('.csv')]
dfs = []
error_log = defaultdict(list)
try:
    for file in files:
        dfs.append(pd.read_csv(root_folder+file, sep='\t', on_bad_lines='warn'))
except pd.errors.EmptyDataError:
    print(f"Error: {file} is empty")
    error_log[file].append("File is empty")
    
except Exception as e:
    print(f"Error loading {file}: {str(e)}")
    error_log[file].append(str(e))

# data = pd.concat([pd.read_csv(root_folder+f, sep='\t') for f in os.listdir(root_folder) if f.endswith('.csv')])

C:\Users\ckunde\AppData\Local\Temp\ipykernel_9784\4162367295.py:8: ParserWarning: Skipping line 4: expected 203 fields, saw 205
Skipping line 5: expected 203 fields, saw 205
Skipping line 6: expected 203 fields, saw 205
Skipping line 7: expected 203 fields, saw 205
Skipping line 8: expected 203 fields, saw 205
Skipping line 9: expected 203 fields, saw 205
Skipping line 10: expected 203 fields, saw 205
Skipping line 11: expected 203 fields, saw 205
Skipping line 12: expected 203 fields, saw 205
Skipping line 13: expected 203 fields, saw 205
Skipping line 14: expected 203 fields, saw 205
Skipping line 15: expected 203 fields, saw 205
Skipping line 16: expected 203 fields, saw 205
Skipping line 17: expected 203 fields, saw 205
Skipping line 18: expected 203 fields, saw 205
Skipping line 19: expected 203 fields, saw 205
Skipping line 20: expected 203 fields, saw 205
Skipping line 21: expected 203 fields, saw 205
Skipping line 22: expected 203 fields, saw 205
Skipping line 23: expected 203 

In [12]:
if dfs:
    combined_df = pd.concat(dfs, ignore_index=True)
    

In [13]:
# columns_to_keep = [
#     'datetime',
#     'Marked barcode',
#     'CJW ST1_3 Gauge value before grease',
#     'CJW ST1_5 Gauge value after grease',
#     'CJW ST1_5 Gauge value difference',
#     'CJG ST1_3 Gauge value before grease',
#     'CJG ST1_5 Gauge value after grease',
#     'CJG ST1_5 Gauge value difference'
# ]
# df = combined_df.drop(columns=[col for col in combined_df.columns if col not in columns_to_keep])

In [14]:
def split_by_middle_number(df: pd.DataFrame) -> dict[str, pd.DataFrame]:
    """
    Split a DataFrame into multiple DataFrames based on the middle number in 'Marked barcode' column.
    
    Args:
        df (pd.DataFrame): Input DataFrame containing a 'Marked barcode' column
        
    Returns:
        dict[str, pd.DataFrame]: Dictionary mapping middle numbers to corresponding DataFrames
        
    Example:
        If 'Marked barcode' contains values like "123 456 789", the middle number "456"
        will be used as the grouping key.
    """
    # Extract middle numbers and ensure it's a 1-dimensional Series
    middle_numbers = (
        df['Marked barcode']
        .str.extract(r'\d+\s+(\d+)\s+\d+')
        .squeeze()
        .fillna('')  # Handle any rows that don't match the pattern
    )
    
    # Create groups directly using the Series
    groups = df.groupby(middle_numbers)
    
    # Return dictionary of DataFrames, excluding empty group if it exists
    return {
        str(name): group.copy()
                       .reset_index(drop=True)
                       .fillna('-') 
        for name, group in groups
        if name != ''  # Skip groups with empty middle numbers
    }

In [15]:
separated_dfs = split_by_middle_number(combined_df)

In [16]:
# Create results directory if it doesn't exist
os.makedirs(f'{root_folder}results', exist_ok=True)

# Process each group
for sep_name, sep_df in separated_dfs.items():
    print(f"Processing {sep_name}...")
    
    # Create Excel writer object
    excel_path = f'{root_folder}results/{sep_name}.xlsx'
    with pd.ExcelWriter(excel_path, engine='openpyxl') as writer:
        # Sort and save CJW sheet
        print(f"Saving CJW sheet for {sep_name}...")
        sep_df.sort_values(['CJW ST1_5 Gauge value after grease', 'datetime'], ascending=[True, True]).to_excel(
            writer, 
            sheet_name='CJW',
            index=False
        )
        
        # Sort and save CJG sheet
        print(f"Saving CJG sheet for {sep_name}...")
        sep_df.sort_values(['CJG ST1_5 Gauge value after grease', 'datetime'], ascending=[True, True]).to_excel(
            writer, 
            sheet_name='CJG',
            index=False
        )
    
    print(f"Completed saving {sep_name}")

Processing 3311011...
Saving CJW sheet for 3311011...
Saving CJG sheet for 3311011...
Completed saving 3311011
Processing 3311012...
Saving CJW sheet for 3311012...
Saving CJG sheet for 3311012...
Completed saving 3311012
Processing 3311013...
Saving CJW sheet for 3311013...
Saving CJG sheet for 3311013...
Completed saving 3311013
Processing 3311014...
Saving CJW sheet for 3311014...
Saving CJG sheet for 3311014...
Completed saving 3311014
Processing 4111009...
Saving CJW sheet for 4111009...
Saving CJG sheet for 4111009...
Completed saving 4111009
Processing 4111010...
Saving CJW sheet for 4111010...
Saving CJG sheet for 4111010...
Completed saving 4111010
